In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
    classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    # config file 들고오기
    cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

    PREFIX = '../../input/data/'
    cfg.runner.max_epochs = 60

  

    # dataset 바꾸기
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = PREFIX
    cfg.data.train.ann_file = PREFIX + 'train.json'
    cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = PREFIX
    cfg.data.val.ann_file = PREFIX + 'val.json'
    cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = PREFIX
    cfg.data.test.ann_file = PREFIX + 'test.json'
    cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.samples_per_gpu = 4

    cfg.optimizer = dict(type='AdamW', lr=0.0001, weight_decay=0.0001)
    cfg.lr_config = dict(
        policy='CosineRestart',
        warmup='linear',
        warmup_iters=200,
        warmup_ratio=1.0 / 10,
        periods=[20,12,12,12,12],
        restart_weights = [1,0.7,0.6,0.5,0.4],
        min_lr_ratio=2e-6)

    cfg.seed= 42
    cfg.gpu_ids = [0]
    cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_coco_mosaic'

    cfg.model.roi_head.bbox_head.num_classes = 11

    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

    #group_name = '아무거나'; ##그룹이름(run 여러개를 그룹으로 묶어줄 수 있습니다)
    project_name = 'stage_3_object_det'; ##프로젝트 이름
    run_name = 'faster_rcnn_r50_fpn_1x_coco_mosaic'; ##run 이름stage_3_object_det
    config_list = {
        'epoch' : cfg.runner.max_epochs,
        'batch_size' :  cfg.data.samples_per_gpu,
        'optimizer' : cfg.optimizer,
        'optimizer_config' : cfg.optimizer_config,
        'lr_config' : cfg.lr_config
    }
    ## 원하는 wandb.init() argument값 추가해주세요
    #cfg.log_config.hooks[1].init_kwargs['group']=group_name   # group name(option)
    cfg.log_config.hooks[1].init_kwargs['project'] = project_name
    cfg.log_config.hooks[1].init_kwargs['name'] = run_name    # run name
    cfg.log_config.hooks[1].init_kwargs['config'] = config_list    # config

In [3]:
model = build_detector(cfg.model)

2021-05-18 04:24:58,779 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-18 04:24:58,780 - mmdet - INFO - Use load_from_torchvision loader
2021-05-18 04:24:59,000 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.21s)
creating index...
index created!


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-18 04:25:06,895 - mmdet - INFO - Start running, host: root@a1eb508df260, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_r50_fpn_1x_coco_mosaic
2021-05-18 04:25:06,896 - mmdet - INFO - workflow: [('train', 1)], max: 60 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=0.74s)
creating index...
index created!
wandb: Currently logged in as: openingsound (use `wandb login --relogin` to force relogin)


2021-05-18 04:25:18,407 - mmdet - INFO - Epoch [1][10/655]	lr: 1.405e-05, eta: 6:03:31, time: 0.555, data_time: 0.225, memory: 2257, loss_rpn_cls: 0.6830, loss_rpn_bbox: 0.0806, loss_cls: 1.5120, acc: 71.7627, loss_bbox: 0.0862, loss: 2.3619, grad_norm: 18.2720


AssertionError: Caught AssertionError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/ml/code/mmdetection_trash/mmdet/datasets/custom.py", line 193, in __getitem__
    data = self.prepare_train_img(idx)
  File "/opt/ml/code/mmdetection_trash/mmdet/datasets/custom.py", line 216, in prepare_train_img
    return self.pipeline(results)
  File "/opt/ml/code/mmdetection_trash/mmdet/datasets/pipelines/compose.py", line 25, in __call__
    data = t(data)
  File "/opt/ml/code/mmdetection